# Heterogenous pure mechanics

We aim to solve
$$\nabla\cdot \sigma = -f$$
with 
$$\sigma = 2\mu \epsilon + \lambda \text{tr}(\epsilon) I$$
where $\epsilon = (\nabla u + (\nabla u)^T)/2$.

The additional feature is that $\mu = \mu(x,y)$ and $\lambda = \lambda (x,y)$. Physically, $\mu$ is the shear modulus (often denoted $G$). $\mu := \sigma_{xy}/\epsilon_{xy}$ (for relatively small strains), where $\sigma_{xy}$ is shear stress, $\epsilon_{xy}$ is shear strain. It measures the stiffness of materials - i.e. the material's response to shear stress. $\lambda$ on the other hand, may be difficult to interpret physically.

In [ ]:
import porepy as pp
import numpy as np
import scipy.sparse as sps

In [ ]:
class Rectangle:
    
    def __init__(self, physdims, foot=np.array([0,0])):
        """ Create a rectangle represented by its vertices in CCW order.
        
        Parameters
        physdims (np.array, (2,) ): Physical dimensions of rectangle
        foot (np.array, (2,) ): Coordinates of SW foot of rectangle. Default: (0,0)
        """
        assert(np.all(np.array(physdims) > 0))
        ft = np.array([foot]).T
        hz = np.array([[physdims[0], 0]]).T # Horizontal shift
        vt = np.array([[0, physdims[1]]]).T # Vertical shift
        self.L = np.hstack((ft, ft+hz))
        self.U = np.add(np.flip(self.L, axis=1), vt)
        self.nodes = np.hstack((self.L, self.U))
        
        # Construction assertion:
        #from porepy.geometry.geometry_property_checks import is_ccw_polygon
        #assert(is_ccw_polygon(self.nodes))
        
        
class RectangleByCenter(Rectangle):
    
    def __init__(self, physdims, center):
        """ Create a rectangle from a center coordinate.
        
        See rectangle for more information.
        
        Parameters
        physdims (np.array, (2,) ): Physical dimensions of rectangle
        foot (np.array, (2,) ): Coordinates of center of rectangle
        
        """
        c = np.array(center); pd = np.array(physdims)
        
        foot = c - pd/2
        super().__init__(physdims, foot)


class Mechanics:
    def __init__(self, mesh_args, folder_name):
        """ Initialize the mechanics class.
        
        Parameters
        mesh_args (dict): Containing mesh sizes.
                For now, this should be e.g. {'n': 5}. 
                We use cartesian meshes.
        folder_name (str): Name of storage folder for visualization.
        
        """
        from porepy.geometry.geometry_property_checks import point_in_polygon
        self.mesh_args = mesh_args
        self.folder_name = folder_name
        
        # Variable
        self.displacement_variable = "u"
        
        # Keyword
        self.mechanics_parameter_keyword = "mechanics"
        
        
    def create_grid(self):
        """ Create a 2D domain without fractures.
        
        The method requires the following attribute:
            mesh_args (dict): Containing the mesh sizes
        
        The following attributes are assigned to self:
            box (dict): Bounding box of domain.
            gb (pp.GridBucket): The grid bucket.
            Nd (int): Dimension of the domain.
        
        """
        n = self.mesh_args['n']
        nx = np.array((n,n))
        self.box = {'xmin': 0, 'ymin': 0, 'xmax': 1, 'ymax': 1}
        
        #g = pp.CartGrid([n,n], physdims=[1,1])
        #g.compute_geometry()
        gb = pp.meshing.cart_grid([], nx, physdims=[1,1])
        gb.compute_geometry()
        gb.assign_node_ordering()
        
        self.gb = gb
        self.Nd = gb.dim_max()
        
    def domain_boundary_sides(self, g):
        """ 
        Obtain indices for faces on each side of the domain boundary.
        
        """
        tol = 1e-10
        box = self.box
        east = g.face_centers[0] > box["xmax"] - tol
        west = g.face_centers[0] < box["xmin"] + tol
        north = g.face_centers[1] > box["ymax"] - tol
        south = g.face_centers[1] < box["ymin"] + tol
        if self.Nd == 2:
            top = np.zeros(g.num_faces, dtype=bool)
            bottom = top.copy()
        else:
            top = g.face_centers[2] > box["zmax"] - tol
            bottom = g.face_centers[2] < box["zmin"] + tol
        
        all_bf = g.get_boundary_faces()
        return all_bf, east, west, north, south
        
    def bc_type(self, g):
        all_bf, *_ = self.domain_boundary_sides(g)
        bc = pp.BoundaryConditionVectorial(g, all_bf, "dir")
        return bc
    
    def bc_values(self, g):
        values = np.zeros((self.Nd, g.num_faces))
        values = values.ravel('F')
        return values

    def source(self, g):
        return np.zeros(g.dim * g.num_cells)
    
    def set_parameters(self):
        """ Set default parameters to the matrix
        
        The following attributes are assigned to self:
            lam (np.ndarray, (g.num_cells,)): lambda parameter
            mu (np.ndarray, (g.num_cells,)): mu parameter.
            data (pp.Parameters): All parameter data related to the problem.

        """
        for g, d in self.gb:
            if g.dim == self.Nd:
                
                # Rock parameters
                lam = np.ones(g.num_cells)
                mu = np.ones(g.num_cells)
                C = pp.FourthOrderTensor(mu, lam)
                self.lam = lam
                self.mu = mu
                
                # Boundary condition setup 1:
                # 0 dirichlet everywhere. Neumann push down on top.
                all_bf, east, west, north, south = domain_boundary_sides(g)
                
                
                # Boundary conditions and source value
                bc = self.bc_type(g)
                bc_val = self.bc_values(g)
                f = self.source(g)

                specified_parameters = {'fourth_order_tensor': C, 'source': f, 
                                        'bc': bc, 'bc_values': bc_val}
                pp.initialize_default_data(g, d, self.mechanics_parameter_keyword, 
                                           specified_parameters)
        
    def insert_heterogenity(self, physdims, center, mu, lam, tol=1e-8):
        """ Insert a heterogenity to the domain.

        A heterogenity in this sense means a rectangle with
        a different mu and/or lambda (e.g. different C) than
        the surrounding matrix.
        Assumes that parameters are already been set. Assumes existense
        of attributes self.lam and self.mu

        Parameters:
        physdims (np.ndarray, (2,) ): Physical dimensions of rectangle
        center (np.ndarray, (2,)): Coordinates of center of rectangle
        mu (float): Value of mu in the rectangle.
        lam (float): Value of lambda in the rectangle.
        tol (float, Default: 1e-8): Tolerance of rectangle bounds.

        The following attributes in self are updated:
            mu (np.ndarray, (g.num_cells,)): mu parameter
            lam (np.ndarray, (g.num_cells,)): Lambda parameter

        In addition, the parameter dictionary for the domain of highest dimension
        (we assume no fracs) has its entry 'fourth_order_tensor' updated

        """
        g = self.gb.grids_of_dimension(self.Nd)[0]
        data = self.gb.node_props(g)
        R = RectangleByCenter(physdims, center)

        # This code assumes 2D.
        nds = np.copy(g.cell_centers)
        nds[2] = np.array(range(g.num_cells)) # Use z-coordinate as index.
        rect_nodes = nds[2, point_in_polygon(R.nodes, nds[[0,1],:], tol=tol)].astype(int)

        self.mu[rect_nodes] = mu
        self.lam[rect_nodes] = lam
        C = pp.FourthOrderTensor(mu, lam)

        data[pp.PARAMETERS]['fourth_order_tensor'] = C
        
    def assign_variables(self):
        """ Assign variables to the node of the grid bucket.
        
        We currently assume no fractures.
        
        """
        gb = self.gb
        for g, d in gb:
            if g.dim == self.Nd:
                d[pp.PRIMARY_VARIABLES] = {
                    self.displacement_variable: {"cells": self.Nd}
                }
            else:
                d[pp.PRIMARY_VARIABLES] = {}
        
        for e, d in gb.edges():
            
            if e[0].dim == self.Nd:
                pass
            
            d[pp.PRIMARY_VARIABLES] = {}
    
    def assign_discretization(self):
        """ 
        Assign discretization to the node of the grid bucket
        
        """
        # We solve the highest dimension problem using mpsa.
        # There should be no other dimensions present.
        Nd = self.Nd
        gb = self.gb
        mpsa = pp.Mpsa(self.mechanics_parameter_keyword)
        
        for g, d in gb:
            if g.dim == Nd:
                d[pp.DISCRETIZATION] = {
                    self.displacement_variable: {'mpsa': mpsa}}
        
        # Assume no edges, so we don't loop over them.



def run_mechanics(setup):
    """
    Method for solving linear elasticity on a non-fractured (currently 2D) domain.
    
    Arguments:
        setup: A setup class with methods:
                create_grid(): Create and return the grid bucket
                set_parameters(): assigns data to grid bucket.
                assign_variables(): assigns variables on grid bucket nodes and edges.
                assign_discretizations(): assigns discretizations on grid bucket nodes
                    and edges.
            and attributes:
                folder_name: returns a string. The data from the simulation will be
                written to the file 'folder_name/' + setup.out_name and the vtk files to
                'res_plot/' + setup.out_name

    """
    # Define the grid without overwriting an existing one.
    if "gb" in setup.__dict__:
        gb = setup.gb
    else:
        gb = setup.create_grid()
        gb = setup.gb
    
    # Set parameters, variables and discretizations on grid.
    setup.set_parameters()
    setup.assign_variables()
    setup.assign_discretization()
    
    # Setup assembler and discretize
    assembler = pp.Assembler(gb)
    assembler.discretize()
    
    
    g = gb.grids_of_dimension(2)[0]
    data = gb.node_props(g)
    viz = pp.Exporter(g, name="mechanics", folder=setup.folder_name)
    
    # Solve the system
    A, b = assembler.assemble_matrix_rhs()
    solution = sps.linalg.spsolve(A, b)
    assembler.distribute_variable(solution)
    
    u2 = data[pp.STATE][setup.displacement_variable]
    viz.write_vtk({"ux": u2[::2], "uy": u2[1::2]})

In [ ]:
mesh_args = {'n': 5}
folder_name = "heterogenous_test"

mech = Mechanics(mesh_args, folder_name)

u2 = run_mechanics(mech)

In [ ]:
gb = mech.gb
g = gb.grids_of_dimension(2)[0]
all_bf, east, west, north, south = mech.domain_boundary_sides(g)

In [ ]:
all_bf

In [ ]:
east

In [ ]:
west